In [73]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.neighbors import KNeighborsRegressor
import plotly.graph_objects as go
import random
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import os
import re
import warnings

In [74]:
test=pd.read_csv("spotify_top_charts_22.csv")
train=pd.read_csv("spotify_dataset.csv")
bigdata=pd.read_csv("charts.csv")

Extracting the year from the date from bigdata (the dataset with Spotify URL's)

In [79]:
year=[]
for i in range(0,len(bigdata)):
    year.append(int(bigdata['date'][i][0:4]))
bigdata['year']=year

Subsetting data from only the US bigdata (the dataset with Spotify URL's)

In [80]:
USdata=bigdata[bigdata['region']=='United States']

In [81]:
os.environ['SPOTIPY_CLIENT_ID']='bf70bf6d43e44bbeaca6aa7d724aa18d'
os.environ['SPOTIPY_CLIENT_SECRET']='b1b9fd3f9fc94b97a1fc9978c4725334'

In [82]:
USdata=USdata.reset_index().drop(columns='index')

# Working with Test set

Subsetting the observatons of bigdata that are present in the test set. Making sure that the track names and artist names align

In [84]:
bigdata_test = bigdata[bigdata['title'].isin(list(test['track_name']))].reset_index().drop(columns='index')

In [85]:
# len(bigdata_test['title'].unique())

In [86]:
bigdata_test_2 = bigdata_test[bigdata_test['artist'].isin(test['artist_names'])]

Getting the title, artist, and url of each song present in the bigdata_test_2 set

In [124]:
usedata = bigdata_test_2.drop_duplicates(subset=['title', 'artist'])

In [128]:
# No NA url's
usedata[usedata['url'].isna()]

In [138]:
finaldata=pd.DataFrame()

for i in range(0,len(usedata)):
    auth_manager=SpotifyClientCredentials()
    sp = spotipy.Spotify(auth_manager=auth_manager)
    url = sp.audio_features(usedata.iloc[i]['url'])
    # d = sp.audio_features(usedata['url'][i])
    title = usedata.iloc[i]['title']
    artist = usedata.iloc[i]['artist']
    df=pd.DataFrame(url)
    df['title'] = title
    df['artist'] = artist
    finaldata=finaldata.append(df)

In [ ]:
# Joining finaldata and test (so test can have all the variables needed)

In [148]:
testingdata = pd.merge(test, finaldata , how='inner', left_on=['artist_names','track_name'], right_on = ['artist','title'])

In [149]:
# No NA's
# testingdata[testingdata['title'].isna()]

# Working with Training set

Subsetting the observatons of bigdata that are present in the training set. Making sure that the track names and artist names align

In [194]:
bigdata_train = bigdata[bigdata['title'].isin(list(train['Song Name']))].reset_index().drop(columns='index')

In [197]:
bigdata_train_2 = bigdata_train[bigdata_train['artist'].isin(train['Artist'])]

Getting the title, artist, and url of each song present in the bigdata_train_2 set

In [157]:
usedata_2 = bigdata_train_2.drop_duplicates(subset=['title', 'artist'])

In [159]:
# No NA url's
usedata_2[usedata_2['url'].isna()]

,title,rank,date,artist,url,region,chart,trend,streams,year


In [178]:
finaldata_2=pd.DataFrame()

# FIRST PART
for i in range(0, len(usedata_2)):
    auth_manager=SpotifyClientCredentials()
    sp = spotipy.Spotify(auth_manager=auth_manager)
    url = sp.audio_features(usedata_2.iloc[i]['url'])
    # d = sp.audio_features(usedata['url'][i])
    title = usedata_2.iloc[i]['title']
    artist = usedata_2.iloc[i]['artist']
    df_2=pd.DataFrame(url)
    df_2['title'] = title
    df_2['artist'] = artist
    finaldata_2=finaldata_2.append(df_2)

In [228]:
trainingdata = pd.merge(train,finaldata_2, how='inner', left_on=['Artist','Song Name'], right_on = ['artist','title'])

,Index,Highest Charting Position,Number of Times Charted,Week of Highest Charting,Song Name,Streams,Artist,Artist Followers,Song ID,Genre,...,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,title,artist
0,1,1,8,2021-07-23--2021-07-30,Beggin',"48,633,449",Måneskin,3377762,3Wrjm47oTz2sjIgck11l5e,"['indie rock italiano', 'italian pop']",...,134.002,audio_features,3Wrjm47oTz2sjIgck11l5e,spotify:track:3Wrjm47oTz2sjIgck11l5e,https://api.spotify.com/v1/tracks/3Wrjm47oTz2s...,https://api.spotify.com/v1/audio-analysis/3Wrj...,211560,4,Beggin',Måneskin
1,2,2,3,2021-07-23--2021-07-30,STAY (with Justin Bieber),"47,248,719",The Kid LAROI,2230022,5HCyWlXZPP0y6Gqq8TgA20,['australian hip hop'],...,169.928,audio_features,5PjdY0CKGZdEuoNab3yDmX,spotify:track:5PjdY0CKGZdEuoNab3yDmX,https://api.spotify.com/v1/tracks/5PjdY0CKGZdE...,https://api.spotify.com/v1/audio-analysis/5Pjd...,141806,4,STAY (with Justin Bieber),The Kid LAROI
2,3,1,11,2021-06-25--2021-07-02,good 4 u,"40,162,559",Olivia Rodrigo,6266514,4ZtFanR9U6ndgddUvNcjcG,['pop'],...,166.928,audio_features,4ZtFanR9U6ndgddUvNcjcG,spotify:track:4ZtFanR9U6ndgddUvNcjcG,https://api.spotify.com/v1/tracks/4ZtFanR9U6nd...,https://api.spotify.com/v1/audio-analysis/4ZtF...,178147,4,good 4 u,Olivia Rodrigo
3,4,3,5,2021-07-02--2021-07-09,Bad Habits,"37,799,456",Ed Sheeran,83293380,6PQ88X9TkUIAUIZJHW2upE,"['pop', 'uk pop']",...,126.026,audio_features,6PQ88X9TkUIAUIZJHW2upE,spotify:track:6PQ88X9TkUIAUIZJHW2upE,https://api.spotify.com/v1/tracks/6PQ88X9TkUIA...,https://api.spotify.com/v1/audio-analysis/6PQ8...,231041,4,Bad Habits,Ed Sheeran
4,5,5,1,2021-07-23--2021-07-30,INDUSTRY BABY (feat. Jack Harlow),"33,948,454",Lil Nas X,5473565,27NovPIUIRrOZoCHxABJwK,"['lgbtq+ hip hop', 'pop rap']",...,150.087,audio_features,5Z9KJZvQzH6PFmb8SNkxuk,spotify:track:5Z9KJZvQzH6PFmb8SNkxuk,https://api.spotify.com/v1/tracks/5Z9KJZvQzH6P...,https://api.spotify.com/v1/audio-analysis/5Z9K...,212353,4,INDUSTRY BABY (feat. Jack Harlow),Lil Nas X
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1549,1552,195,1,2019-12-27--2020-01-03,New Rules,"4,630,675",Dua Lipa,27167675,2ekn2ttSfGqwhhate0LSR0,"['dance pop', 'pop', 'uk pop']",...,116.073,audio_features,2ekn2ttSfGqwhhate0LSR0,spotify:track:2ekn2ttSfGqwhhate0LSR0,https://api.spotify.com/v1/tracks/2ekn2ttSfGqw...,https://api.spotify.com/v1/audio-analysis/2ekn...,209320,4,New Rules,Dua Lipa
1550,1553,196,1,2019-12-27--2020-01-03,Cheirosa - Ao Vivo,"4,623,030",Jorge & Mateus,15019109,2PWjKmjyTZeDpmOUa3a5da,"['sertanejo', 'sertanejo universitario']",...,152.370,audio_features,2PWjKmjyTZeDpmOUa3a5da,spotify:track:2PWjKmjyTZeDpmOUa3a5da,https://api.spotify.com/v1/tracks/2PWjKmjyTZeD...,https://api.spotify.com/v1/audio-analysis/2PWj...,181930,4,Cheirosa - Ao Vivo,Jorge & Mateus
1551,1554,197,1,2019-12-27--2020-01-03,Havana (feat. Young Thug),"4,620,876",Camila Cabello,22698747,1rfofaqEpACxVEHIZBJe6W,"['dance pop', 'electropop', 'pop', 'post-teen ...",...,104.988,audio_features,1rfofaqEpACxVEHIZBJe6W,spotify:track:1rfofaqEpACxVEHIZBJe6W,https://api.spotify.com/v1/tracks/1rfofaqEpACx...,https://api.spotify.com/v1/audio-analysis/1rfo...,217307,4,Havana (feat. Young Thug),Camila Cabello
1552,1555,198,1,2019-12-27--2020-01-03,Surtada - Remix Brega Funk,"4,607,385","Dadá Boladão, Tati Zaqui, OIK",208630,5F8ffc8KWKNawllr5WsW0r,"['brega funk', 'funk carioca']",...,154.064,audio_features,5F8ffc8KWKNawllr5WsW0r,spotify:track:5F8ffc8KWKNawllr5WsW0r,https://api.spotify.com/v1/tracks/5F8ffc8KWKNa...,https://api.spotify.com/v1/audio-analysis/5F8f...,152784,4,Surtada - Remix Brega Funk,"Dadá Boladão, Tati Zaqui, OIK"


In [235]:
trainingdata.to_csv('trainingdata.csv')
testingdata.to_csv('testingdata.csv')